<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-Data-Pre-Processing" data-toc-modified-id="Setup-Data-Pre-Processing-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Setup Data Pre-Processing</a></span><ul class="toc-item"><li><span><a href="#Raw-Data-to-Feature-Data" data-toc-modified-id="Raw-Data-to-Feature-Data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Raw Data to Feature Data</a></span></li><li><span><a href="#Tensor-Data-Structure-Input" data-toc-modified-id="Tensor-Data-Structure-Input-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Tensor Data-Structure Input</a></span></li></ul></li><li><span><a href="#Architecture-Basics" data-toc-modified-id="Architecture-Basics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Architecture Basics</a></span><ul class="toc-item"><li><span><a href="#Gated-CNN" data-toc-modified-id="Gated-CNN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Gated CNN</a></span><ul class="toc-item"><li><span><a href="#GCNN-1D-Time-Series" data-toc-modified-id="GCNN-1D-Time-Series-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>GCNN 1D Time Series</a></span></li><li><span><a href="#GCNN-1D-Residuals" data-toc-modified-id="GCNN-1D-Residuals-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>GCNN 1D Residuals</a></span></li><li><span><a href="#GCNN-2D" data-toc-modified-id="GCNN-2D-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>GCNN 2D</a></span></li></ul></li></ul></li><li><span><a href="#Hyper-Param-Protocol-(hpp)" data-toc-modified-id="Hyper-Param-Protocol-(hpp)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Hyper Param Protocol (hpp)</a></span></li></ul></div>

In [281]:
# Imports e definicoes auxiliares, TODO - fazer module apenas para estas definicoes auxiliares
from IPython.display import Math
#!pip install librosa
import tensorflow as tf
from tensorflow.python import debug as tf_debug
tf.summary.initialize = tf.contrib.summary.initialize
import functools
import numpy as np
import scipy as scp
from scipy import signal, linalg
from numpy import asarray, array, ravel, repeat, prod, mean, where, ones
import librosa
import sys
audio_open = librosa.audio.audioread.audio_open

#!pip install scikits.audiolab
#!pip install --upgrade-strategy=only-if-needed git+https://github.com/Uiuran/BregmanToolkit
#!pip install scikit-image
#!pip install --upgrade-strategy=only-if-needed git+https://github.com/librosa/

import matplotlib.pyplot as plt
from tensorflow.python.client import timeline
correlate = signal.correlate
import os

# Tensorboard display
from IPython.display import clear_output, Image, display, HTML
'''
GATED
CONVOLUTIONAL NEURAL NETWORK Architecture P.O.C (proof of concept)
'''

# Change esdruxulo names given by tensorflow development team
tf.variable_scope = tf.compat.v1.variable_scope

# funções auxiliares
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
# Função que usa HTML e javascript para exibir tensorboar no notebook e web
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.    
    
    Parameters:
    
    - session: The TensorFlow session to be frozen.
    
    Default Keyword Parameters
    
    - keep_var_names: A list of variable names that should not be frozen. Defaults None.     
    - output_names: Names of the relevant graph outputs/operation/tensor to be written. Defaults None.
    - clear_devices: Remove the device directives from the graph for better portability. Defaults True.
    
    return The frozen graph definition.
    """
    
    graph = session.graph
    with graph.as_default():
      freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
      output_names = output_names or []
      output_names += [v.op.name for v in tf.global_variables()]
      # Graph -> GraphDef ProtoBuf
      input_graph_def = graph.as_graph_def()
      if clear_devices:
        for node in input_graph_def.node:
          node.device = ""
      frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                    output_names, freeze_var_names)
      return frozen_graph
    
def save_model(sess, model_name='./model.pb', output_names=True):
    
  frozen_graph = freeze_session(sess, output_names= output_names)
  tf.train.write_graph(frozen_graph, "model", model_name, as_text=False)   
    
def downsampling(data,num):
  '''
   Downsampling por num(int) feito como se fosse no C
  '''
  d = np.copy(data)
  j=0
  for i in range(len(d)):

    if i%int(num) == 0:
      j +=1
    else:
      d[i] = 0.0
  datadown = np.zeros(j)
  j=0
  for i in range(len(d)):
  
    if d[i] != 0.0: 
      #print(d[i])
      datadown[j] = d[i]
      j += 1  

  return datadown

def amostragem(data, fs, plot = False):
  '''
   Amostragem de downsampling de uma serie temporal segundo Nyquist-Shannon. Alem disso o numero maximo
   de frequencias com o spectro calculado por ciclo deve ser igual ao numero de pontos N/2 (principio de incerteza).

   
   argumentos:
   
   - data: serie temporal
   - fs: frequência de amostragem do encode original
   - plot: opcional, plota o espectrograma com parametros padrões para obter máxima frequência maior que a largura de banda.
  '''
 
  f,t,Sxx = signal.spectrogram(data,fs,nfft=fs/4,nperseg=fs/5,noverlap=fs/10,scaling='spectrum',mode='magnitude')
  freq = np.array([i for i in zip(t,Sxx.T) if np.shape(f[i[1]>np.std(i[1])])[0] != 0])

  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13) )
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

  return [[ i[0], np.ceil(np.float(fs)/(2.0*np.max(f[i[1]>np.std(i[1])]) )) ] for i in freq]

def subbed_timeseries(measure,fs):
  '''
   Return Sxx (log squared amplitude of stft spectra), f (frequencies), t (time steps)
  '''
  sampa = np.min(np.array(amostragem(np.array(measure),fs, plot=False))[:,1])  
  fsa = (fs/int(sampa))
    
  a = downsampling(np.array(measure),sampa )

  return a,fsa

def subbed_spect(measure,fs,plot=True):
  '''
   Return Sxx (log squared amplitude of stft spectra), f (frequencies), t (time steps)
  '''
  sampa = np.min(np.array(amostragem(np.array(measure),fs, plot=plot))[:,1])  
  fsa = (fs/int(sampa))
    
  a = downsampling(np.array(measure),sampa )
  f,t,Sxx = signal.spectrogram(a,fsa,nfft=fsa/4,nperseg=fsa/5,noverlap=fsa/10,scaling='spectrum',mode='magnitude')
  #Cut zero variance part of spectra(useless recording)   
  Sxx = Sxx[:,np.var(np.log(Sxx+1e-13),axis=0) > 0.5]
  l = len(Sxx[0,:])
  t = t[:l] 
  
  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13))
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    
  return np.abs(np.log(Sxx+1e-13)),f,t
    
def sample_gaussmix(data=None,shape=(500,), mean_base = 0.0,var_base = 0.15, num_mods = 30, random_mods=False):
  '''
    Generate sample noise from num_mods different gaussian with data or only noise.
    
    TODO: select mod from a list of tuples with (mean,variance)
  '''
  num_mods = int(num_mods)
  if type(data) != type(None):
    shape = np.shape(data)  
  c = np.int(shape[0]/num_mods)
  np.random.set_state = 1440*shape[0]

  if type(data) != type(None):
    if random_mods:
      if len(shape) > 1:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),c) for i in range(num_mods)]).flatten() )    
    else:
      if len(shape) > 1:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (data + np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),c) for i in range(num_mods)]).flatten() )      
  else:
    if random_mods:
      if len(shape) > 1:
        mix = lambda data: (np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (np.array([np.random.normal(mean_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),var_base*np.mod(np.random.randint((i+1),high=1234*i+4),num_mods),c) for i in range(num_mods)]).flatten() )    
    else:
      if len(shape) > 1:
        mix = lambda data: (np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),(c,shape[1:]) ) for i in range(num_mods)]).flatten() )
      else:
        mix = lambda data: (np.array([np.random.normal(mean_base*(i+1),var_base*(i+1),c) for i in range(num_mods)]).flatten() )      
  return mix(data)

#Device function helpers

_conv_dim = lambda x,xx,dilation,stride : np.ceil((x-(xx-1)*dilation)/stride)

def _calculatefilter(initialdim, filterlist):

  a = initialdim  
  b = 0 
  c = 0
  for filter in filterlist:
    a = _conv_dim(a,filter,1,1)
    if a < 0:
      b = a
      c = 1
      break
  if c == 1:
    a = b
  return np.int32(a)

class _dev_selector:
    
  def __init__(self,**kwargs):
    self.arg1 = kwargs['arg1']
    
  def __call__(self,op):
    if op.name.split('/')[0] == 'gcnn2d_d1':
      return '/device:CPU:0'
    elif op.name.split('/')[0] == 'gcnn2d':
      return '/device:CPU:0'
    else:
      return '/device:CPU:0'

  def _conv2D_on_gpu(op):
    if op.type == "Conv2D":
      return "/device:GPU:*"
    else:
      return "/device:CPU:*"

    
# In TensorFlow a device name is a string of the following form:
#   /job:<name>/replica:<replica>/task:<task>/device:<type>:<device_num>
#
# <name> is a short identifier conforming to the regexp
#     [a-zA-Z][_a-zA-Z]*
# <type> is a supported device type (e.g. 'cpu' or 'gpu')
# <replica>, <task>, <device_num> are small non-negative integers and are
# densely allocated (except in tests).
#
# For some purposes, we also allow device patterns, which can specify
# some or none of the specific fields above, with missing components,
# or "<component>:*" indicating "any value allowed for that component.
#
# For example:
#   "/job:param_server"   - Consider any devices in the "param_server" job
#   "/device:cpu:*"       - Consider any cpu devices in any job/task/replica
#   "/job:*/replica:*/task:*/device:cpu:*"  - Consider any cpu devices in any
#                                             job/task/replica
#   "/job:w/replica:0/task:0/device:gpu:*"  - Consider any gpu devices in
#                                             replica 0, task 0, of job "w"

def print_display_cgraph(graph,verbose=False):
  if verbose:
    for op in graph.get_operations():
      print(op.name)
  graph_def = graph.as_graph_def()
  show_graph(graph_def)

#From Danijar Blog decorator for scope definition example, hack/explore decorators    

'''
def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if no arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator
'''
###
## Danijar option to define scope with decorators
#

def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if no arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator
#####
####
##
#

class AttrDict(dict):
    
  __getattr__ = dict.__getitem__
  __setattr__ = dict.__setitem__   
    


# Setup Data Pre-Processing 

## Raw Data to Feature Data

In [79]:
# Raw data to Log-Spectogram 
# Read data from the hard-drive
spect_list = []
raw_list = []
nomes = [f for f in os.listdir('/home/stattus4dpenalva/stattus4/src/')]
label_list = []

for i in range(len(nomes[0:10])):
  #ignore .mat
  if nomes[i].find('.mat') == -1:    
    data,fs = librosa.load('/home/stattus4dpenalva/stattus4/src/'+nomes[i],sr=None)
    spect_list.append( subbed_spect(data,fs,plot=False) )
    raw_list.append((data,fs))
    label_list.append(nomes[i][0:2])

## Tensor Data-Structure Input

In [86]:
# Data to Tensor Data for consumption on Tensorflow models
# TODO - equalize data size for downsampled raw data
# - On supposition that the data size is the same for all time series, if not is necessary to assert it.
# - The same is true for spect, however downsampled spect may lead to smaller ts.
#ts  = sample_gaussmix(shape=(10000,), mean_base=0.5, var_base=0.15, random_mods=True, num_mods=50)

batch = len(spect_list)
data_size_ts = len(raw_list[0][0])
data_size_image = (len(spect_list[0][1]),len(spect_list[0][2]))
channels = 1

data_tensor_1D = np.ndarray( shape=(batch,data_size_ts,channels) )
data_tensor_2D = np.ndarray( shape=(batch,data_size_image[0],data_size_image[1],channels) )

#for i in range(batch):
  #data_tensor_1D[i,:,0] = raw_list[i][0]
  #data_tensor_2D[i,:,:,0] = spect_list[i][0][:,:]

# Architecture Basics

In [416]:
# Definitions: Build computation graph and run session

#####
###  Inicio de refatoracao usando principios de design patterns. 
# TODO: pesquisar mais e mais e mais a fundo o design do Tensorflow pra seguir de perto.

from enum import Enum

class BlockTypes(Enum):
  GCNN2D = 'gcnn2d'
  SOFTMAX = 'softmax'
  CROSSENTROPY = 'loss-crossentropy'
  INPUT = 'signal_in'
  REDUCEMEAN = 'reducemean'
  
class ArchBlocks:

  def __add__(self, block):
    pass
    
class Architecture(ArchBlocks):
  '''
    aa
  '''
  def __init__(self, **kwargs):
    
    self.modules = {'0':[ArchBlocks.INPUT,0,0,[],[]]}

  def __add__(self, archblock1):
    pass
    #self.modules = archblock1
    
    
  def _block_specification(self,key):    
    pass

class Stattus4NeuralNetAPI:
    
  def __init__(self, datatype, datasocket = False, **kwargs):
    '''
     Input
     
     datatype: Audio, Pressure, Flow.
     datasocket: if True tries to fetch data from a database and push output to database 
     pre-configured through kwargs, if config is not provided then tries to read from datafeeder or dump to local file-system.   
     
     Optional
     num_of_channels: number of channels for Audio data, if not given get only the first audio and discard the others.    
    '''
    self.datatype = datatype
    self.datasocket = datasocket
    
  def setup_hyperparameters(self, architectures):
    '''
      Input
      
      architectures: Architecture object specifying archetypical structure to be investigated for its hyperparams.

      Return a dictionary of the given 
    '''
    pass

#######
#######
 ####
  ### 
   #
    
def _get_tensor_list_from_scopes(graph, tensor_scope_name):
    
    names = []
    for i in range(len(tensor_scope_name)):
      names.append(graph.get_tensor_by_name(tensor_scope_name[i]+':0'))
    if len(names) > 1:      
      return tf.keras.layers.concatenate( names, axis = 1)
    else:
      return names[0]

def _find_softmax(graph, **kwargs):

  names = []    
  for op in graph.get_operations():
     
    name = op.name
   
    if name.split('/')[0][0:7] == 'softmax':

      a = name.find('softmax')                
      if name.split('/')[1] == 'transpose_1':
        names.append(op.name)
      
  return names
# provisional class name for further development in the future as Archtecture block class unleashs

#  FUTURE: Name/Var Scope handler. This class/template must be used to implement logic of names 
# according to the architecture block connection. E.G. if a block is just the same from the other in the
# sequence them it must be named as if is a deep part of the same architecture blocks. If it is a ramification
# of the given below but the same arch them he must be named with another tag.
# When duplicating some block to architectonic use, E.G. if you need to frame your data or feature, them each
# frame must have an coeherent name space block, like the today ordimatlicly done for repeated name space block,
# but this must be automaticly done for identical built blocks of the model, without the need to explicity code it.
# Names must be given as unique ID according to te input-output architecture connection, if it has the same IO from previous Block in the sequence,
# then each Input-ARchBlock-Output that is IDentical must have the same name with the appropriated tag for deepnes in the sequence of blocks

# Automatic update of the name scopes according to the architecture.
# Dont change Identical sequential IO Name spaces


  

class Builder:  
  
  def __init__(self, **kwargs):
    '''
      Build a Tensorflow computational graph, from scratch.      
        
      Requires signal input parameters such:
      - batch: for batch size
      - datasize: give a tuple with the signal input shape
      - channels: self explain.
      
      
      Note 0: You can always call Builder.get_directives(archblocktype) to know the inner params to the building block architecture.
      Note 1: The object guard reference for each op in a dictionary namescopes with the name of each block.
      TODO - export serialized file containing the model (e.g. .pb)
    '''  
    
    #Use attribute dict ##FUTURE: use arch block class with enums specifing the blocks
    
    #Provisional name tracking for this network.
    
    self.namescopes = AttrDict()    
    self.architectures = AttrDict()
    self.architectures.gcnn1d = self._gcnn1d
    self.architectures.gcnn2d = self._gcnn2d
    self.architectures.reducemean = self._reducemean
    self.architectures.losscrossentropy = self._losscrossentropy
    self.architectures.softmax = self._softmax
    self.architectures.residualgcnn1d = self._residualgcnn1d
    
    
    #TODO also use attribute dict, each arch has it own directive dict in with each directive has its shape has value
    ##FUTURE: use arch block class with enums specifing the blocks
    ##   Auto compile directives from json and query parameters,
    ## return requisition or error if parameter not given or is not from appropriated shape and type. 
    self.directives = {
        'gcnn2d': {'channels_out':1,
                   'filter_size':2},
        'reducemean':{'None':None},
        'losscrossentropy':{'num_labels':1,
                            'batch':1,
                            'learningrate':1},
        'softmax':{'num_labels':1},  
        'residualgcnn1d':{'channels_out':1,
                          'filter_size':1},
        'gcnn1d':{'channels_out':1,
                   'filter_size':2
                 },
        'signalimg':{'batch':1,
                   'datasize':2,
                   'channels':1,
                   'dtype':1
                    }    
    }
    
    tf.reset_default_graph()
    # TODO-Define Specific Exception Handler for the Input
    try:
        
      self.batch = kwargs['batch']
      self.datasize = kwargs['datasize']
      self.channels = '' or kwargs['channels']
      self.dtype = kwargs['dtype']
    
    except KeyError:
        
      sys.exit('Signal Input Not Defined Error')      
    
    try:
        
      self.num_input = kwargs['num_input']
    
    except:
        
      self.num_input = 1
      
    self.graph = tf.Graph()
    self.signal_in = []  
    # FUTURE: this will go to another class
    self.archstack = []    
    
  def __call__(self, **kwargs):        
   
    try:
      isinput = kwargs['isinput']
      if type(isinput) != type(bool()):
        isinput = True
    except:
      isinput = False
    
    try:
      lastnamescope= kwargs['lastnamescope']
      if type(lastnamescope) != type(bool()):
        lastnamescope= False
    except:
      lastnamescope= True       


    if isinput:
      with self.graph.as_default():
      # TODO - Different input shapes and data types (char,string,hexa,float,int...)                      
        if self.num_input > 0:
          self.signal_in.append(tf.placeholder(self.dtype,shape=(self.batch,self.datasize[0],self.datasize[1],self.channels),name='signal_in'))           
          self.namescopes['signal_in'] = self.signal_in
          self.num_input += 1
        
      return self.signal_in[-1]
    elif lastnamescope:
      with self.graph.as_default():  
        return self.graph.get_tensor_by_name(self.graph.get_operations()[-1].name+':0')
    else:
      try:
        namescope = kwargs['namescope']
        with self.graph.as_default():
          with tf.name_scope(namescope+'/'):
            opa = 0
            for op in self.graph.get_operations():
              if op.name.split('/')[0] == namescope:
                opa = op 
            return self.graph.get_tensor_by_name(opa.name+':0')
      except:        
        sys.exit("Namescope not given to the input of architectures")            
        
  def assertive(self, archname):
    if hasattr(self, 'deepness') != True or getattr(self, 'deepness') == None:
      raise NameError('{} deepness not setted'.format(archname))
    
    if hasattr(self, 'numblock') != True or getattr(self, 'numblock') == None:
      raise NameError('{} block number not setted'.format(archname))    

  def _gcnn2d(self, **kwargs):
    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']
          
    except KeyError:
        
      sys.exit('Parameters Not Defined Error')  
    
    signal_in = self(**kwargs)
     
    ####            
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias                    
    self.assertive('gcnn2d')
    
    with self.graph.as_default():
      with tf.variable_scope('gcnn2d'+self.numblock+self.deepness[0]):
        
        self.namescopes['gcnn2d'+self.numblock+self.deepness[0]] = []    
        self.archstack.append( 'gcnn2d'+self.numblock+self.deepness[0] )
        
        with self.graph.device(_dev_selector(arg1='foo')):
          conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True, kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in)
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(conv_linear)            

        with self.graph.device(_dev_selector(arg1='foo')):
          conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in),name='conv_sigmoid')          
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(conv_gate)  
            
        with self.graph.device(_dev_selector(arg1='foo')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions') 
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(gated_convolutions)
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
    
  def _gcnn1d(self, **kwargs):

    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']

    except Exception:
        
      sys.exit('Parameters Not Defined Error')
     
      signal_in = self(**kwargs)
        
    # postfix = self.get_namepostfix('gccn1d',**kwargs)
      with self.graph.as_default():
        with tf.variable_scope('gccn1d'):#+postfix):
                             
          with self.graph.device(_dev_selector(arg1='foo')):
            conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(signal_in)
        
          with self.graph.device(_dev_selector(arg1='foo')):
            conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(signal_in),name='conv_sigmoid')
        
          with self.graph.device(_dev_selector(arg1='foo')):
            gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions') 

  def _residualgcnn1d(self, **kwargs):  
    
    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']
        
    except KeyError:
        
      sys.exit('Parameters Not Defined Error')  
  
    signal_in = self(**kwargs)
                               
    
      ####
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
                             
        # postfix = self.get_namepostfix('gccn1d',**kwargs)
    with self.graph.as_default():
      with tf.variable_scope('residualgccn1d'):#+postfix):                            
                    
        with self.graph.device(_dev_selector(arg1='foo')):            
          conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(signal_in)
        with self.graph.device(_dev_selector(arg1='foo')):        
          conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(signal_in),name='conv_sigmoid')
        with self.graph.device(_dev_selector(arg1='foo')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
        
      # Input channels must be the same size of the convolution channels output (ie number of filters applied)
      with self.graph.device(_dev_selector(arg1='foo')):
        residual = tf.add(gated_convolutions,signal_in,name='residual')        

  def _softmax(self, **kwargs):        

    try:        
      num_labels = kwargs['num_labels']      
    except Exception:        
      sys.exit('Parameters Not Defined Error')       
    
    signal_in = self(**kwargs)    
    self.assertive('softmax')   
        
    with self.graph.as_default():
      with tf.variable_scope('softmax'+self.numblock+self.deepness[0]):
        
        self.namescopes['softmax'+self.numblock+self.deepness[0]] = []          
        self.archstack.append('softmax'+self.numblock+self.deepness[0])
        
        with self.graph.device(_dev_selector(arg1='foo')):
          logits = tf.contrib.layers.fully_connected(signal_in, num_labels, activation_fn=None, normalizer_fn=None, normalizer_params=None, weights_initializer=tf.initializers.lecun_normal(seed=731), weights_regularizer=None, biases_initializer=tf.initializers.lecun_normal(seed=777), biases_regularizer=None, reuse=None, variables_collections=None, outputs_collections=None, trainable=True, scope='logit')
          self.namescopes['softmax'+self.numblock+self.deepness[0]].append(logits)
            
        with self.graph.device(_dev_selector(arg1='foo')):
          softmax = tf.nn.softmax(logits,axis=0)            
          self.namescopes['softmax'+self.numblock+self.deepness[0]].append(softmax)
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None

  def _losscrossentropy(self, **kwargs):    
    
    try:        
      num_labels = kwargs['num_labels']
      batch = kwargs['batch']   
      learningrate = kwargs['learningrate']
    except Exception:        
      sys.exit('Parameters Not Defined Error')
    # TODO SIGNAL IN de acordo com arquitetura.
    signal_in = self(**kwargs)            
    ####        
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias   
    self.assertive('losscrossentropy') 
    
    with self.graph.as_default():
      with tf.variable_scope('losscrossentropy'+self.numblock+self.deepness[0]):        

        self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]] = []        
        self.archstack.append('losscrossentropy'+self.numblock+self.deepness[0] )
        self.l_vec = tf.placeholder(tf.float32,(batch,num_labels), name='labels')

        with self.graph.device(_dev_selector(arg1='foo')):
          loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0)(self.l_vec, signal_in)         
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(loss)
          self.loss = loss

        with self.graph.device(_dev_selector(arg1='foo')):
          self.minimize_op = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(loss)
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(self.minimize_op)
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
        
  def _reducemean(self, **kwargs):
     
    scope_tensor_name = _find_softmax(self.graph)
    inputs = _get_tensor_list_from_scopes(self.graph, scope_tensor_name)    
    
    self.assertive('reducemean')
    
    with self.graph.as_default():
      with tf.variable_scope('reducemean'+self.numblock+self.deepness[0]):

        self.namescopes['reducemean'+self.numblock+self.deepness[0]] = []
        self.archstack.append( 'reducemean'+self.numblock+self.deepness[0] )
  
        with self.graph.device(_dev_selector(arg1='foo')):
          rm = tf.math.reduce_mean( inputs, axis=1 )
          self.namescopes['reducemean'+self.numblock+self.deepness[0]].append(rm)
          rmshape = tf.shape(rm)
          rmean = tf.reshape(rm, shape=(rmshape[0], rmshape[2]))   
          self.namescopes['reducemean'+self.numblock+self.deepness[0]].append(rmean)
            
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
  
  def get_directives(self,arch):
      
    # TODO: Use as automatic requirement builder from file, maybe json.
    # Provisory setup for namespace tags
    #self._nametags = {arch:{}}
    return self.directives[arch]     
      
  # provisory configuration for next sequence of build module names, FUTURE: Hyper-Param-Protocol will learn
# the names automatically from given configuration or from the graph architecture updating.
  def config_block_name(self,deepness=0,numblock=0):
    '''
      Helper function to configure block architecture names.
      Inputs are deepness of the block, if there is any hidden layer, and the block number if there is more than one block with this architecture in the graph.          
    '''
    if deepness < 0:
      deepness = 0
      self.deepness = ['']
    else:  
      self.deepness = ['']
      for i in range(deepness):
        self.deepness.append('_d'+str(i))
    if numblock <= 0:
      self.numblock = ''
    else:
      self.numblock = '_'+str(numblock)
        
  # Definitions: Build computation graph and run session
  def build_graph_module(self, arch, show_cgraph=True, verbose=False, **kwargs):
    '''
      Build a Tensorflow computational graph.building_function
      Currently supports 1-D gcnn or residual 1-D gcnn for unidimensional variable and 2D gcnn for bidimensional frame data.
    
      tensor_scope_name:  name scope of the architecture unit to build the next module
    
        
      kwargs:    
      filter_size: conv filter size. Scalar number for time series, bidimensional tuple for image.
      channels_out: Number of channels for output convolution.
    
      TODO - export serialized file containing the model (e.g. .pb)
    '''
    
    self.architectures[arch](**kwargs) 
    
    with self.graph.as_default(): 
      if show_cgraph:
        print_display_cgraph(self.graph, verbose=verbose)
        
  def redefine_block(self,blockname):
    
    tf.reset_default_graph()
    graph = tf.Graph()
    with graph.as_default():
                              
      tf.graph_util.import_graph_def(self.graph.as_graph_def(), name=blockname)
      its = self.namescopes.items()
      self.namescopes = {}
      self.namescopes[blockname] = {}
                              
      for k,v in its:
        if k != 'signal_in':
          self.namescopes[blockname][k] = v
                              
      del self.graph
      self.graph = graph

  # Build block from name in the top level of the blocks, this is a previous version that take names inside namescope dict and build new block  
  def from_block(self,archblockname,tag = ""):
    with self.graph.as_default():
      print(self.namescopes)
      opa = self.namescopes[archblockname][self.archstack[-1]]            
      tf.contrib.copy_graph.copy_op_to_graph(opa[-1].op, self.graph, [], scope=archblockname+tag)
    
    
  def run_cgraph(self, feed_dict, op_to_run = '', number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True):
    '''
     Run Session
     TODO - session log output
     TODO - in the mode feed forward, only get tensor value for input, in the mode minimize run the last op that represents minimization     
     TODO - make op to run optional depending on modes
     TODO - getvariable by namescope
    '''       
    if new_session:
      session = tf.Session(graph=self.graph)  
      session = tf_debug.TensorBoardDebugWrapperSession(session,"grpc://localhost:6064")
    else:
      session = tf.get_default_session()
    
    with session.as_default():
    #with tf.device(_dev_selector(''))
      feed_dict = feed_dict
      options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
      run_metadata = tf.RunMetadata()
      opval = []
      lossval = np.zeros(number_of_runs)
    
    # Search for Initialization: TODO binary search, using brute force ... 
      ops = self.graph.get_operations()
      opinit = ops[-1]
      if opinit.name[-4:] != 'init':
        c = 0  
        for op in ops:
          if op.name[-4:] == 'init':
            session.run(op)
            print('Modulo Variables Initialized {}'.format(c) )
            c += 1
        if c == 0:
          with self.graph.as_default():
            tf.global_variables_initializer()
            session.run(self.graph.get_operations()[-1])
            print('Global Variables Initialized')
    
      else:
        session.run(opinit)
        print('Global Variables Initialized')
        
    # Perform  gradient descent steps or feed forward according to the Operation Required   
      
      self.op_to_run = op_to_run
      with tf.summary.FileWriter('/home/penalvad/stattus4/stattus4-audio-models/notebooks/',graph=self.graph,session=session) as writer:
        tf.summary.initialize(graph=self.graph,session=session)

             
        for step in range(number_of_runs):
        
          if mode == 'minimize':
            loss_value,_ = session.run([self.loss,self.minimize_op], feed_dict=feed_dict)
            lossval[step] = loss_value
            if step % 1 == 0 and verbose:        
              print("Step:", step, " Loss:", lossval[step])           

              if step % 5 == 0 and step != 0:          
                loss_diff = np.diff(lossval[np.nonzero(lossval)])
                print("Mean Loss Growth ", np.mean(loss_diff) )        
            
          if self.op_to_run != '':            
            op_value = session.run(self.op_to_run, feed_dict=feed_dict)             
            opval.append( op_value )

      #gated_convs_value = session.run(graph.get_tensor_by_name('gated_convolutions:0'), feed_dict = feed_dict, options=options, run_metadata=run_metadata)
    return opval

##  Gated CNN
  Gated CNN is a doubled CNN in whom one of the convoluted signals does the role of opening/closing the network, giving an **Attention Mechanism** to the convolution, for being activated by a sigmoid.
  It gives non-vanishing gradient, since the multiplication rule for the derivative applies, also, applies gradient to the linear convoluted part.
  

### GCNN 1D Time Series
  In time series version, since the desirable learning is based on **past** events, or you cannot uphold the assumption that you have acess to future data, have to make sure that the convolution is **causal**, that is
\begin{align}
y_{n}= a_{i}x_{n-i}=a_{n-j}x_{j}
\end{align} 
  Giving at last, if the filter has length k, k-1 zero padding to the input x.

In [28]:
#build graph
graph = build_graph( (None,l[0],channels), arch = 'gcnn1d',print_ops = True, new_graph=True, show_cgraph = True)
'''
obs: if you have a graph sometimes is necessary to run more than one time wit reset_default_graph to get a new graph
'''

NameError: name 'l' is not defined

In [96]:
# Run the constructed graph

feed_dict = {
  graph.get_tensor_by_name(graph.get_operations()[-1].name+':0'):data_tensor
}

op_value = run_cgraph(graph, graph.get_operations()[-1].name+':0', feed_dict, number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True)
print(op_value)

('Step:', 0, ' Loss:', array([[[ 0.7076101 ,  0.75356215,  0.44474468,  0.89668995],
        [ 0.404285  ,  0.10061911,  0.8874869 ,  0.22097981],
        [-0.05257223,  0.8174497 ,  0.00637231,  0.00501409],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]],
      dtype=float32))
('Step:', 1, ' Loss:', array([[[ 0.7076101 ,  0.75356215,  0.44474468,  0.89668995],
        [ 0.404285  ,  0.10061911,  0.8874869 ,  0.22097981],
        [-0.05257223,  0.8174497 ,

/home/stattus4dpenalva/miniconda2/envs/scicomptf_37/lib/python2.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/stattus4dpenalva/miniconda2/envs/scicomptf_37/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### GCNN 1D Residuals

In [32]:
#build graph
graph = build_graph( (None,l[0],channels), arch = 'residual gcnn1d',print_ops = True,new_graph=True, show_cgraph = True)

NameError: name 'l' is not defined

In [31]:
# Run the constructed graph

feed_dict = {
  graph.get_tensor_by_name(graph.get_operations()[-1].name+':0'):data_tensor
}

op_value = run_cgraph(graph, graph.get_operations()[-1].name+':0', feed_dict, number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True)
print(op_value)

KeyError: "The name 'init:0' refers to a Tensor which does not exist. The operation, 'init', exists but only has 0 outputs."

### GCNN 2D
  This unit uses sigmoid function in all frame pixels and do element-wise multiplication. Use it for images or time-series TF Features frame. The standard convolution mode is Valid. The output size of the convolution is given by:
 
 Padding Valid
\begin{align}
dimemsion output size = \ceil{ \frac{dim size - (kernel dim size - 1)*dilation rate}{stride} }
\end{align} 

Where dilation rate is how may pixel are each filter spaced from each other, when dilation rate >= 1 then stride = 1 (since if you stride will be losing information from the data).

In [418]:
#build graph
# gcnn2d,softmax,losscrossentropy
#config1 = [ {"0_frames":4,"0_0":(10,30,40,2)}, {"1_1":(10,2,2,8), "1_2":(10,2,1,16), "1_3":(10,4,1,32), "1_4":(10,8,1,64), "1_5":(10,8,1,64)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]
buildgraph = Builder(dtype=tf.float32,batch=10,datasize=(40,30), num_input = 100, channels=3)

buildgraph.get_directives('gcnn2d')
buildgraph.get_directives('softmax')
#buildgraph.set_archname('frame')
buildgraph.get_directives('reducemean')
buildgraph.get_directives('losscrossentropy')
flist = [4,4,4,8,8,8]

buildgraph.config_block_name(deepness=7,numblock=0)
buildgraph.build_graph_module('gcnn2d', channels_out = 8,filter_size=(4,4), isinput=True, show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 16,filter_size=(4,4), show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 32,filter_size=(4,4), lastnamescope=True, namescope='gcnn2d',verbose=True, show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 64,filter_size=(8,8), show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 128,filter_size=(8,8), show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 256,filter_size=(8,7), show_cgraph=False)
a = _calculatefilter(40, flist)
#b = _calculatefilter(30, flist)

buildgraph.build_graph_module('gcnn2d', channels_out = 512,filter_size=(a,1), isinput=False, lastnamescope=True, show_cgraph=False)
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('softmax', num_labels=2, isinput=False, lastnamescope=True, show_cgraph=True)
#refatorar essa funcao redefine block em funcao da outra se possivel, from block
buildgraph.redefine_block('block')
buildgraph.from_block('block',tag = "_1")
buildgraph.from_block('block',tag = "_2")
buildgraph.from_block('block',tag = "_3")
print_display_cgraph(buildgraph.graph)

{'block': {'gcnn2d': [<tf.Tensor 'gcnn2d/conv_linear/BiasAdd:0' shape=(10, 37, 27, 8) dtype=float32>, <tf.Tensor 'gcnn2d/conv_sigmoid:0' shape=(10, 37, 27, 8) dtype=float32>, <tf.Tensor 'gcnn2d/gated_convolutions:0' shape=(10, 37, 27, 8) dtype=float32>], 'gcnn2d_d2': [<tf.Tensor 'gcnn2d_d2/conv_linear/BiasAdd:0' shape=(10, 24, 14, 64) dtype=float32>, <tf.Tensor 'gcnn2d_d2/conv_sigmoid:0' shape=(10, 24, 14, 64) dtype=float32>, <tf.Tensor 'gcnn2d_d2/gated_convolutions:0' shape=(10, 24, 14, 64) dtype=float32>], 'gcnn2d_d3': [<tf.Tensor 'gcnn2d_d3/conv_linear/BiasAdd:0' shape=(10, 17, 7, 128) dtype=float32>, <tf.Tensor 'gcnn2d_d3/conv_sigmoid:0' shape=(10, 17, 7, 128) dtype=float32>, <tf.Tensor 'gcnn2d_d3/gated_convolutions:0' shape=(10, 17, 7, 128) dtype=float32>], 'gcnn2d_d0': [<tf.Tensor 'gcnn2d_d0/conv_linear/BiasAdd:0' shape=(10, 34, 24, 16) dtype=float32>, <tf.Tensor 'gcnn2d_d0/conv_sigmoid:0' shape=(10, 34, 24, 16) dtype=float32>, <tf.Tensor 'gcnn2d_d0/gated_convolutions:0' shape=(1

In [360]:
buildgraph.config_block_name(deepness=7,numblock=1)
buildgraph.build_graph_module('gcnn2d', channels_out = 8,filter_size=(4,4), isinput=True, input_num = 2, show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 16,filter_size=(4,4), show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 32,filter_size=(4,4), lastnamescope=True, namescope='gcnn2d',verbose=True, show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 64,filter_size=(8,8), show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 128,filter_size=(8,8), show_cgraph=False)
buildgraph.build_graph_module('gcnn2d', channels_out = 256,filter_size=(8,7), show_cgraph=False)
a = _calculatefilter(40, flist)
#b = _calculatefilter(30, flist)
buildgraph.build_graph_module('gcnn2d', channels_out = 512,filter_size=(a,1), lastnamescope=True, show_cgraph=False)
buildgraph.config_block_name(deepness=0,numblock=1)
# Fazer função para reduzir 1 dimensão do shape se tamanho for igual a 1
buildgraph.build_graph_module('softmax', num_labels=2, isinput=False, lastnamescope=True, show_cgraph=False)
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('reducemean', isinput=False, lastnamescope=True, show_cgraph=False)
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('losscrossentropy', num_labels = 2, batch=10, learningrate=0.005, isinput=False, lastnamescope=True, show_cgraph=True)

In [285]:
data_tensor = [np.random.rand(10,40,30,3), np.random.rand(10,40,30,3)]
lvec = [[1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1]]
print(lvec)
feed_dict = {
  buildgraph.signal_in[0]:data_tensor[0],
  buildgraph.signal_in[1]:data_tensor[1],
  buildgraph.l_vec:lvec    
}

output = buildgraph.run_cgraph(feed_dict, op_to_run = buildgraph.namescopes.gcnn2d_1[1], number_of_runs = 10, mode = 'minimize', new_session=True, output_log = True, verbose=True)
print('Tensor {}'.format(buildgraph.namescopes.gcnn2d_1[1].name))
print(np.shape(output))
print(output)

[[1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1]]

SIGINT received. Quit program? (Y/n): Y


SystemExit: 0

/home/penalvad/miniconda3/envs/scicomptf1_37/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Hyper Param Protocol (hpp)

Two interfaces implementing the following:

A - Space x Selection Algorithm x Architecture

- Implementations of this interface specifies the search space for the Deep Learning model, including space size i.e. the number of points and space complexity that is objects to data dict (Variability)/structure/number of sources/Unique Ids (e.g. location). The Space Complex objects should be implement through his own interface.

B - Space Complex

- Central to space complexity are the specifications of the already given above. Once one have implemented the specification than can implement the Interface itself. 

In [579]:
#   HPP Definitions. First DISS  implementation by the already made architecture, brute force selection Algol, 
# and Space == Space Size.
# TODO - refatorar.
# TODO - full implementation

class Hpp:
  '''
   Hyperparam tuning protocol.
   
   Input
   - space: Space object giving the size and the complexity of the space
   - selection_algorithm: function running algorithm that search through space using specific evaluation (also implemented in the algorithm)
   - architecture: Architecture object specifying archetipical architecture to be tuned by searching in space.
  '''

  def __init__(self,space = {},selection_algorithm = lambda x: 'foo',architecture='gcnn2d'):
    
    self.space_size = space.get_size()
    self.data_structure = space.get_complex().data.get_structure()    
    self.source_id = space.get_complex().get_ids()
    self.data_dict = space.get_complex().data.get_dict()
    self.algorithm = selection_algorithm
    self.architecture = architecture
    
  def _parse_config(config_list):
    '''
      Receives a config and build from dict to perform  examples:
      config0 = [ {"0_frames": lambda : 4 1 if 10 == 1 else 0,"0_0": lambda :(10,30,40,2), "0_1": lambda :(10,2,2,8), "0_2": lambda :(10,2,1,16), "0_3": lambda :(10,4,1,32), "0_4": lambda :(10,8,1,64), "0_5": lambda :(10,16,1,128), "0_6": lambda :(10,13,1,256)},{"1_labels": lambda :2},{"2_null": lambda :'a'},{"3_labels": lambda :2,"3_learningrate": lambda :0.005}]
      config1 = [ {"0_frames":4,"0_0":(10,30,40,2)}, {"1_1":(10,2,2,8), "1_2":(10,2,1,16), "1_3":(10,4,1,32), "1_4":(10,8,1,64), "1_5":(10,8,1,64)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]
      config2 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,2,1,16), "0_3":(10,4,1,32), "0_4":(10,8,1,64) },{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config3 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,2,1,16), "0_3":(10,4,1,32) },{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config4 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,2,1,16)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config5 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      
      config6 = [{"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,6,1,16), "0_3":(10,6,1,32), "0_4":(10,6,1,64), "0_5":(10,10,1,128), "0_6":(10,16,1,256)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]
      config7 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,6,1,16), "0_3":(10,6,1,32), "0_4":(10,6,1,64), "0_5":(10,10,1,128)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config8 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,6,1,16), "0_3":(10,6,1,32), "0_4":(10,6,1,64)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config9 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,6,1,16), "0_3":(10,6,1,32)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config10 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8), "0_2":(10,6,1,16)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]      
      config11 = [ {"0_frames":4,"0_0":(10,30,40,2), "0_1":(10,2,2,8)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]
    '''    
    overall_config = []
    overall_attr = []
    overall_deepness = []
    for block_config in config_list:        
      layer_configs = []        
      layer_attr = []
      layer_deep = 0
      # attributes       
      bc = block_config.items()
      for item in bc:
        if item[0].split('_')[1].isalnum():
          layer_configs.append(item)
          layer_deep += 1
        else:
          layer_attr.append(item)
      
      layer_configs.sort()
      layer_attr.sort()
      
      overall_deepness.append(layer_deep)
      overall_config.append( dict(layer_configs) )
      overall_attr.append( dict(layer_attr) )
    
    return overall_attr,overall_config,overall_deepness 
    
  def _is_vec_embed():
    pass

  def run(self):    
    '''
    try: 
      assert type(tf.get_default_graph()) == type(tf.Graph())
      graph = tf.get_default_graph()
    except AssertionError:
      graph = tf.Graph()         
    ''' 
      
    tf.reset_default_graph()
    graph = tf.Graph()
    s = 0
    for config in range(self.algorithm.size):

      s += 1
      layer_config_list = self.algorithm()
      overall_attr,overall_config,overall_deepness = _parse_config(layer_config_list)
      
      with tf.variable_scope('Config {}'.format(s)):                
        
        #TODO Assert that it is a block 0 and its attributes (for this case only nframe)            
        
        blockattr = overall_attr.pop(0)
        nframe = blockattr.values()[0]        
        config = overall_config.pop(0)     
        
        for blocks in range(len(self.architecture)):            
          
          graph = build_graph_module(graph, scope_tensor_name=op.name, arch = self.architecture[blocks], print_ops = True, name_scope = True, show_cgraph = True, filter_size=(k[1],k[2]), channels_out = k[3], deepness = '_d1',num_labels=2,learning_rate=0.005)
          op = graph.get_operations()[-1]
        
          try:
            blockattr = overall_attr[0]            
            if blockattr.keys()[0].split('_')[0] == blocks:
              blockattr = overall_attr.pop(0)
              for attr,v in blockattr:
                if attr.split('_')[1] == 'frames':
                  nframes
                
          except:
            pass                             
          
        for k in layer_config[2:]:

            
            
            
            '''
        names = []
for op in graph.get_operations():
    
  if op.name.split('/')[1] == 'transpose_1' and op.name.split('/')[0][0:7] == 'softmax':
    names.append(op.name)    

graph = build_graph_module(graph, scope_tensor_name=names, arch = 'reducemean',print_ops = True, name_scope = True, show_cgraph = True, filter_size=(2,1), channels_out = 64, deepness = '',num_labels=4,learning_rate=0.005, verbose=True)
op = graph.get_operations()[-1]
graph = build_graph_module(graph, scope_tensor_name=op.name, arch = 'loss-crossentropy',print_ops = True, name_scope = True, show_cgraph = True, filter_size=(2,1), channels_out = 64, deepness = '',num_labels=4,learning_rate=0.1, batch=40, verbose=True)
    
    layer_config = self.algorithm()
    with tf.variable_scope('secondparam'):
        
      graph = build_graph( (layer_config[0][0],layer_config[0][1],layer_config[0][2],layer_config[0][3]), arch = self.architecture,print_ops = True,new_graph=False, show_cgraph = True, filter_size=(layer_config[1][1],layer_config[1][2]), channels_out = layer_config[1][3])
      op = graph.get_operations()[-1]
      for k in layer_config[2:]:
        graph = build_graph_module(graph, scope_tensor_name=op.name, arch = self.architecture, print_ops = True, name_scope = True, show_cgraph = True, filter_size=(k[1],k[2]), channels_out = k[3], deepness = '_d1',num_labels=2,learning_rate=0.005)
        op = graph.get_operations()[-1]

      '''
    
    
    
    
    
class Space:
  '''
   Space is built upon space size and complexity.
     
   Input:
   - size: an integer, the number of configurations to look for the model.      
   - spacecomplex: Unique Ids, a hashable type or string specifing source e.g. location. A SigmaComplex object specifying meta-data, i.e. data configurations and 
   possibly other relevant keys to hyper-param searching such features, diversity mensures and so on.
   SigmaComplex must be an interface to be implemented according to the data specification, possibly separating mutable from immutable characters.    
  '''    
    
  def __init__(self, size, spacecomplex):
    self.size = size
    self.sigmacomplex = spacecomplex
        
  def get_size(self):
    return self.size
    
  def get_complex(self):
    return self.sigmacomplex
        
  def get_ids(self):
    return  self.sigmacomplex.get_ids()
    
  def get_sourcenum(self):
    return self.sigmacomplex.data.sourcenum

  def get_dict(self):
    return self.sigmacomplex.data.get_dict()

  def get_structure(self):
    return self.sigmacomplex.data.get_structure()

class SigmaComplex:
  ''' 
      Unique Ids, a hashable type or string specifing source e.g. location. A SigmaComplex object specifying meta-data, i.e. data configurations and 
     possibly other relevant keys to hyper-param searching such features, diversity mensures and so on.
     SigmaComplex must be an interface to be implemented according to the data specification, possibly separating mutable from immutable characters.    
     
     Data specification and ids will correspond to the empirical collected data according to the geo-located point in a device that has known inner workings
     according to the laws of physics.
  '''
  def __init__(self, ids, data):
    self.ids = ids
    self.data = data   
    
    
  def get_ids(self):
    return  self.ids

  def get_dict(self):
    return self.data.get_dict()

  def get_structure(self):
    return self.data.get_structure()

class DataMeta:
  '''
   Data MaTter specification according to device collector, its inner workings (laws of physics), and other relevant keys.
  '''  
  def __init__(self):
    
    self.structure = "image"
    self.data_dict = { "labels" : ["cv","sv"] , "features" : ["spect","framed"], "channel_num": 1 }
    
  def get_dict(self):
    
    return self.data_dict
    
  def get_structure(self):
    
    return self.structure

In [581]:
# Example of 'foo'

spacecomplex = SigmaComplex(["Chala-head-chala"] , DataMeta())
space = Space( 5, spacecomplex)
hyperparam = Hpp(space, lambda x: '飛輪功', "gcnn2d")

In [598]:
#  Build search space data structure. To be added to SigmaComplex class/interface/template
#  "softmax","reducemean","loss-crossentropy"

config0 = [ {"0_frames":4,"0_0":(10,30,2,2), "0_1":(10,2,2,8), "0_2":(10,2,1,16), "0_3":(10,4,1,32), "0_4":(10,8,1,64), "0_5":(10,16,1,128), {"1_labels":2} ]
config1 = [ {"0_frames":4,"0_0":(10,30,2,2), "0_1":(10,2,2,8), "0_2":(10,2,1,16), "0_3":(10,4,1,32)} ]
config2 = [ {"0_frames":4,"0_0":(10,30,2,2), "0_1":(10,2,2,8) ]
config3 = [ {"0_frames":4,"0_0":(10,30,2,2), "0_1":(10,4,2,8), "0_2":(10,4,1,16), "0_3":(10,4,1,32), "0_4":(10,4,1,64), "0_5":(10,7,1,128), "0_6":(10,7,1,256), "0_7":(10,7,1,512)}, {"1_labels":2},  ]
search_space = ( config0,  config1) 

In [582]:
# hash( '''( ( {"0_sld":4},{"0_0":(10,30.0,40.0)}),({"1_sld":3},{"1_0":(10,30.0,40.0,40.0)} ) )''' )
# Example exhaustive search (河南科技大学)
# you can use yield if you want to
class search(object):
  def __init__(self, listconfig):
    self.size = len(listconfig)
    self.num = 0
    self.config = listconfig
   
  def __iter__(self):
    return self
   
       # Python 3 compatibility
  def __next__(self):
    return self.next()
   
  def next(self):
    if self.num < self.size:
      cur, self.num = self.config[self.num], self.num+1
      return cur
    else:
      raise StopIteration()
  def __call__(self):
    return self.next()
       
spacecomplex = SigmaComplex(["Chala-head-chala"] , DataMeta())
space = Space( 5, spacecomplex)
#TODO implementation of the Space->SigmaComplex->DataMeta()
hyperparam = Hpp(space, search( search_space ), ["gcnn2d","softmax","reducemean","loss-crossentropy"])

In [583]:
hyperparam.run()